# Setup
Setup for getting VRM-E and propensity scores, etc.

In [2]:
import itertools
import sklearn
import pandas as pd
import json
import numpy as np
import ast
from tqdm.auto import tqdm
import statistics
import math
import csv
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn.calibration import calibration_curve
from copy import deepcopy
import pprint
import ast
import statistics
import math
import time
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    mean_squared_error,
    r2_score,
    roc_auc_score,
    average_precision_score,
)
import matplotlib.pyplot as plt
from nltk import agreement
import scipy.stats as stats
import seaborn as sns
np.random.seed(1)

/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## VRM-E

In [43]:
embedding_path = '../data/raw/df_embeddings.csv'
submission_path = '../data/raw/df_submission_rating.csv'


df_embeddings = pd.read_csv(embedding_path)
df_embeddings = df_embeddings.T
df_embeddings.columns=df_embeddings.iloc[0]
df_embeddings = df_embeddings.iloc[1: , :]


tqdm.pandas()
def get_numpy (row):
  return ast.literal_eval(row.embedding)

df_embeddings['embedding'] = df_embeddings.apply(lambda x: get_numpy(x), axis =1)

df_submissions = pd.read_csv(submission_path)
df_submission_labels = df_submissions[['id','title','conf_year','keywords','AVG_rating']]

df_embeddings_2017 = df_embeddings.merge(df_submission_labels[df_submission_labels['conf_year']==2017], left_on='paper_id',right_on='id')
df_embeddings_2018 = df_embeddings.merge(df_submission_labels[df_submission_labels['conf_year']==2018], left_on='paper_id',right_on='id')
assert df_submission_labels[df_submission_labels['conf_year']==2017].shape[0] == df_embeddings_2017.shape[0]

#Section 3.3 Step 2 agglomerative clustering on cosine distance
x = np.array(df_embeddings_2017.embedding.tolist())
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.1, linkage="average", metric = 'cosine').fit(x)
df_embeddings_2017['agg_cluster'] = clustering.labels_.tolist()
x = np.array(df_embeddings_2017.embedding.tolist())
y = np.array(df_embeddings_2017.agg_cluster.tolist())
clf = NearestCentroid()
clf.fit(x, y)

assert df_embeddings_2018.shape[0] == df_submission_labels[df_submission_labels['conf_year']==2018].shape[0]

#Section 3.3. Step 3

#setting up KNN for 2018
neigh = NearestNeighbors( n_neighbors=10, metric = 'cosine', radius = 0.3)
non_anchor_embedding_2018 = np.array(df_embeddings_2018.embedding.to_list())
neigh.fit(non_anchor_embedding_2018)

#setting up closest centroid for anchor group 2017
anchor_embedding_2017 = np.array(df_embeddings_2017.embedding.tolist())
anchor_agg_clusters_2017 = np.array(df_embeddings_2017.agg_cluster.tolist())
clf = NearestCentroid()
clf.fit(anchor_embedding_2017, anchor_agg_clusters_2017)



#dictionary of all the agg clusters and the 10 KNN from 2018
dict_agg_cluster_matches ={}
tuning_param_cos_dist = 0.3
for cluster_id in np.unique(clustering.labels_):
    
    distances, indices = neigh.kneighbors([clf.centroids_[cluster_id]])
    df_anchor_embedding = pd.concat([pd.DataFrame(data = distances.T,columns =['cos_dist']),pd.DataFrame(indices.T,columns=['indices'])],axis=1)

    #get all the specified cosine distance 2018 papers
    #tuple of (dataframe of 2018 matched papers, cosine distances)
    dict_agg_cluster_matches[cluster_id] = (
        df_embeddings_2018.iloc[df_anchor_embedding[df_anchor_embedding['cos_dist']<= tuning_param_cos_dist].indices.to_list(), :],
        df_anchor_embedding[df_anchor_embedding['cos_dist']<= tuning_param_cos_dist].cos_dist.to_list()
    )


def lambda_get_2018_matches(row):
    #get embedding matches from 2018 papers
    #returning relevant information
    df_clustered_papers = dict_agg_cluster_matches[row.agg_cluster]
    
    lst_paper_titles = df_clustered_papers[0].title.tolist()
    lst_paper_ids = df_clustered_papers[0].paper_id.tolist()
    ls_paper_keywords = df_clustered_papers[0].keywords.values.tolist()
    ls_cos_distances = df_clustered_papers[1]
    
    return lst_paper_titles, ls_paper_keywords, lst_paper_ids, ls_cos_distances

df_embeddings_2017[['titles_2018','keywords_2018','id_2018','cos_dist_2018']]= df_embeddings_2017.apply(lambda x: lambda_get_2018_matches(x),axis=1, result_type ='expand')
df_cos_dist_sample = pd.concat([df_embeddings_2017[['agg_cluster']],pd.DataFrame(df_embeddings_2017["cos_dist_2018"].to_list())], axis=1)

#tuning charts
#get the max number of samples with the lowest possible cosine distance
df_tuning = df_cos_dist_sample.drop(['agg_cluster'],axis = 1)

data = []
for tuning_param_cos_dist in np.linspace(0,0.3 ,1000):
    input_row = {}
    sample_number_from_2017 = df_tuning[df_tuning<=tuning_param_cos_dist].any(axis=1).sum()
    
    input_row['sample_number_2017'] = sample_number_from_2017
    input_row['cosine_distance'] = tuning_param_cos_dist
    if sample_number_from_2017 == df_tuning.shape[0]:
        input_row['full_sample_flag'] = 'Full Sample'
    else:
        input_row['full_sample_flag'] = 'Partial Sample'
    data.append(input_row)


x = pd.DataFrame(data)

#Tune KNN matching based on smallest possible cosine distance

tuning_param_knn = 10
tuning_param_cos_dist = 0.234535

neigh = NearestNeighbors( n_neighbors=tuning_param_knn, metric = 'cosine', radius = 0.3)
non_anchor_embedding_2018 = np.array(df_embeddings_2018.embedding.to_list())
neigh.fit(non_anchor_embedding_2018)

#setting up closest centroid for anchor group 2017
anchor_embedding_2017 = np.array(df_embeddings_2017.embedding.tolist())
anchor_agg_clusters_2017 = np.array(df_embeddings_2017.agg_cluster.tolist())
clf = NearestCentroid()
clf.fit(anchor_embedding_2017, anchor_agg_clusters_2017)


#dictionary of all the agg clusters and the 20 KNN from 2018
dict_agg_cluster_matches ={}
for cluster_id in np.unique(clustering.labels_):
    
    distances, indices = neigh.kneighbors([clf.centroids_[cluster_id]])
    df_anchor_embedding = pd.concat([pd.DataFrame(data = distances.T,columns =['cos_dist']),pd.DataFrame(indices.T,columns=['indices'])],axis=1)

    #get all the specified cosine distance 2018 papers
    #tuple of (dataframe of 2018 matched papers, cosine distances)
    dict_agg_cluster_matches[cluster_id] = (
        df_embeddings_2018.iloc[df_anchor_embedding[df_anchor_embedding['cos_dist']<= tuning_param_cos_dist].indices.to_list(), :],
        df_anchor_embedding[df_anchor_embedding['cos_dist']<= tuning_param_cos_dist].cos_dist.to_list()
    )

def lambda_get_2018_matches(row):
    #get embedding matches from 2018 papers
    #returning relevant information
    df_clustered_papers = dict_agg_cluster_matches[row.agg_cluster]
    
    lst_paper_titles = df_clustered_papers[0].title.tolist()
    lst_paper_ids = df_clustered_papers[0].paper_id.tolist()
    ls_paper_keywords = df_clustered_papers[0].keywords.values.tolist()
    ls_cos_distances = df_clustered_papers[1]
    
    return lst_paper_titles, ls_paper_keywords, lst_paper_ids, ls_cos_distances
    
    
    

df_embeddings_2017[['titles_2018','keywords_2018','id_2018','cos_dist_2018']]= df_embeddings_2017.apply(lambda x: lambda_get_2018_matches(x),axis=1, result_type ='expand')

def get_num_knn_matches(row):
    count = len(row.titles_2018)
    return(len(row.titles_2018))

df_embeddings_2017['num_knn_matches'] = df_embeddings_2017.apply(lambda x: get_num_knn_matches(x),axis =1)

assert df_embeddings_2017.shape[0] == df_submission_labels[df_submission_labels['conf_year']==2017].shape[0]

#Matching potential outcome estimator
#Keith et al. 2020
#https://aclanthology.org/2020.acl-main.474.pdf
#equation 7 and 8

def lambda_get_match_potential_outcomes(row):
    #equation 7 in the paper
    paper_ids = row.id_2018
    big_m = len(row.id_2018)
    if big_m == 0:
        return None
    ratings = [df_embeddings_2018[df_embeddings_2018['paper_id'] == paper_id].AVG_rating.values[0] for paper_id in paper_ids]
    return sum(ratings)/big_m
        
    

df_embeddings_2017['match_ave_rating'] = df_embeddings_2017.apply(lambda row: lambda_get_match_potential_outcomes(row), axis =1)
df_embeddings_2017['diff_2018_2017'] = df_embeddings_2017['match_ave_rating'] - df_embeddings_2017['AVG_rating']

df_embeddings_2017 = df_embeddings_2017.loc[df_embeddings_2017['match_ave_rating'].notnull(),]

assert df_embeddings_2017['match_ave_rating'].shape[0] == df_embeddings_2017['AVG_rating'].shape[0]


## SPSM

In [33]:
df_embeddings = pd.read_csv(embedding_path)
df_embeddings = df_embeddings.T
df_embeddings.columns=df_embeddings.iloc[0]
df_embeddings = df_embeddings.iloc[1: , :]

def get_numpy (row):
  return ast.literal_eval(row.embedding)

df_embeddings['embedding'] = df_embeddings.apply(lambda x: get_numpy(x), axis =1)
df_submissions = pd.read_csv(submission_path)
df_submission_labels = df_submissions[['id','title','conf_year','keywords','AVG_rating']]
df_embeddings = df_embeddings.merge(df_submission_labels[df_submission_labels['conf_year'].isin([2017,2018])], left_on='paper_id',right_on='id')
df_embeddings['treatment'] = df_embeddings['conf_year'].replace([2017,2018],[1,0])

np_embeddings =np.array([np.array(embedding) for embedding in df_embeddings.embedding.to_list()])
paper_id = np.array(df_embeddings.id.to_list())
treatment_id = np.array(df_embeddings.treatment.to_list())

if np_embeddings.shape[0] == treatment_id.shape[0] == df_embeddings.shape[0] == paper_id.shape[0]:
    
    E= np_embeddings
    T = treatment_id
    n_total = df_embeddings.shape[0]
    docids = paper_id
    
    if E.shape[0] == T.shape[0] == n_total == docids.shape[0]:
        print('pass')
    else:
        print('error')
    
else:
    print('error')
    
NUM_CROSSFIT_SPLITS = 2
NUM_CROSSVAL_SPLITS = 4 
RANDOM_STATE = 42 #for replication 

CLASSIFIER = Pipeline([(
            "model",
            LogisticRegression(
                l1_ratio=0.1,
                solver="saga",
                max_iter=20000,
                tol=1e-3, 
                penalty="elasticnet",
                dual=False,
                class_weight="balanced",
                random_state=42,
                ),),])

CLASSIFIER_GRID = {
    "model__C": [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1],
}

#Note: May take awhile to run
prob_t_training = np.array([np.nan]*n_total) #only for reporting training acc numbers 
prob_t_inference = np.array([np.nan]*n_total) #array to save predicted propensity scores 
# Crossfitting 

start = time.time()

crossfit_split = list(StratifiedKFold(n_splits=NUM_CROSSFIT_SPLITS, shuffle=True, random_state=RANDOM_STATE).split(E, T))
for crossfit_number, (train_inds, test_inds) in enumerate(crossfit_split):
    #print('Crossfit num=', crossfit_number)
    # Training 
    # Cross validation for the training split 
    inner_cv = StratifiedShuffleSplit(
                n_splits=NUM_CROSSVAL_SPLITS,
                test_size=1/NUM_CROSSVAL_SPLITS,
                random_state= RANDOM_STATE,
            )
    
    t_model_gridsearch = GridSearchCV(
                        estimator=deepcopy(CLASSIFIER),
                        param_grid=deepcopy(CLASSIFIER_GRID),
                        cv=inner_cv,
                        scoring="roc_auc",
                        refit=True,
                    )
    
    t_model_gridsearch.fit(E[train_inds], T[train_inds])
    prob_t_training[train_inds] = t_model_gridsearch.predict_proba(E[train_inds])[:, 1]
        
    # Inference
    # Probability of T=1
    prop_t = t_model_gridsearch.predict_proba(E[test_inds])[:, 1]
    prob_t_inference[test_inds] = prop_t

end = time.time()

#Subtract Start Time from The End Time
total_time = end - start
assert np.mean(np.isnan(prob_t_training)) == 0.0
assert np.mean(np.isnan(prob_t_inference)) == 0.0
# Training metrics 
pred_model_report_out = {}

def mean_predictions(dummy, y_pred):
    """
    Helpful for error diagnosing. Returns the mean of the predcted values
    Args:
        - dummy : we need this arg so that this function looks the same as
        sklearn error metrics that take inputs y_true, y_pred
        - y_pred : np.array
    """
    return np.mean(y_pred)

def calibration_rmse(y_true, y_pred): 
    """
    Calculates calibration root mean squared error (RMSE). 
    Calibration is the extent to which a model's probabilistic predictions match their 
    corresponding empirical frequencies. 
    See Nguyen and O'Connor 2015's introduction and definitions 
    https://www.emnlp2015.org/proceedings/EMNLP/pdf/EMNLP182.pdf
    """
    prob_true, prob_pred = calibration_curve(y_true, y_pred, n_bins=10, strategy='uniform')
    rms = mean_squared_error(prob_true, prob_pred, squared=False) #False returns RMSE vs MSE 
    return rms


def mean_truth(y_true, dummy):
    """
    Helpful for error diagnosing. Returns the mean of the true values
    Args:
        - y_true : np.array
        - dummy : we need this arg so that this function looks the same as
        sklearn error metrics that take inputs y_true, y_pred
    """
    return np.mean(y_true)

# these classification metrics need the "hard" predictions, e.g. y=[0, 0, 1, ...]
class_hard_name2metric_func = {
    "f1": f1_score,
    "acc": accuracy_score,
    "mean_hard_pred": mean_predictions,
    "mean_true": mean_truth,  # should be same for hard or soft
}

# these classification metrics need the "score" predictions, e.g. y=[0.6, 0.77, 0.2, ...]
class_scores_name2metric_func = {
    "roc_auc": roc_auc_score,
    "ave_prec": average_precision_score,
    "calibration_rmse": calibration_rmse,
    "mean_soft_pred": mean_predictions,
    "mean_true": mean_truth,  # should be same for hard or soft
}

#hard classifications
t_pred_hard = (prob_t_training > 0.5).astype(int)

assert t_pred_hard.shape == prob_t_training.shape == T.shape

str_sep = "--"
for metric_str, metric_func in class_hard_name2metric_func.items():
    pred_model_report_out["treatment_model" + str_sep + metric_str] = metric_func(T, t_pred_hard)
for metric_str, metric_func in class_scores_name2metric_func.items():
    pred_model_report_out["treatment_model" + str_sep + metric_str] = metric_func(T, prob_t_training)

NUM_STRATA = 5 
def get_strata(prob_t_score, num_strata): 
    """
    Reports the strata given the probability score 
    
    Strata index starts at 1 
    """
    strata = np.arange(0, 1.0, 1.0/num_strata)
    return np.digitize(prob_t_score, strata, right=False)
docid2strata = {}
for docid, prob_t_score in zip(docids, prob_t_inference): 
    strata_inferred = get_strata(prob_t_score, NUM_STRATA)
    docid2strata[docid] = (strata_inferred, prob_t_score)
data = []
for x in docid2strata.items():
    input_row = {}
    input_row['paperID'] = x[0]
    input_row['strata'] = x[1][0]
    input_row['pscore'] = x[1][1]
    data.append(input_row)




df_paper_strata = df_embeddings.merge(pd.DataFrame(data), left_on='paper_id',right_on='paperID')[['paper_id','title','strata','keywords','pscore','treatment','AVG_rating','conf_year']]

def strip_char(row):
    word_list = ast.literal_eval(row.keywords)
    s = ''
    for word in word_list:
        s += " " + word.replace(' ','_').replace('-','_').lower()
    return s

df_paper_strata['transformed_keywords'] = df_paper_strata.apply(lambda x: strip_char(x), axis = 1)



boostrap_SPSM = []
for n in range(5001):
    sample = df_paper_strata[df_paper_strata['treatment'] ==1].sample(n=490, replace = True ,random_state=n)

    strata_2_weight = sample[(sample['strata'] ==2) & (sample['treatment'] ==1)].shape[0]/490

    strata_3_weight = sample[(sample['strata'] ==3) & (sample['treatment'] ==1)].shape[0]/490

    strata_4_weight = sample[(sample['strata'] ==4) & (sample['treatment'] ==1)].shape[0]/490

    strat_2_treated = sample[(sample['strata'] ==2) & (sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_2_control = df_paper_strata[(df_paper_strata['strata'] ==2) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()
    
    
    strat_3_treated = sample[(sample['strata'] ==3) & (sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_3_control = df_paper_strata[(df_paper_strata['strata'] ==3) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()
    
    strat_4_treated = sample[(sample['strata'] ==4) & (sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_4_control = df_paper_strata[(df_paper_strata['strata'] ==4) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()

    

    ATT = (strat_4_treated - strat_4_control)* strata_4_weight + (strat_3_treated - strat_3_control)* strata_3_weight + (strat_2_treated - strat_2_control)* strata_2_weight
    boostrap_SPSM.append(ATT)

print('Done')

pass


# Bootstrap with NLP test for statistical significance between methods

## VRM-E vs Naive

In [31]:
#Step 1: Generate bootstrap samples from 2017 data
#step 2: Get ATC from bootstrap samples for VRM-E
#step 3: Get ATC for naive bootstrap: Average(Boot_strap 2017 samples) - Average(2018 Empirical Samples
#step 4: Follow figure 1 from https://aclanthology.org/D12-1091.pdf
#Delta(Bootstrap) = ATC_VRM-E(Bootstrap) - ATC_naive(Boostrap)
#Delta(X) = ATC_VRM-E  - ATC_Naive = -0.17 - -0.25 = 0.08

average_rating_2018 = df_embeddings_2018.AVG_rating.mean()
delta_x = 0.08
s = 0
B = 5000
for n in range(B):
    bootstrap_sample = df_embeddings_2017.sample(n=df_embeddings_2017.shape[0], replace = True, random_state=n)
    ATC_VRM_E = bootstrap_sample.diff_2018_2017.mean()

    ATC_Naive = average_rating_2018 - bootstrap_sample.AVG_rating.mean()

    delta_bootstrap = ATC_VRM_E - ATC_Naive

    if delta_bootstrap > 2*delta_x:
        s += 1

print('P-Value for VRM-E vs SPSM is: ',s/B)

P-Value is:  0.0


## VRM-E vs SPSM

In [ ]:
#Step 1: Generate bootstrap samples from 2017 data
#step 2: Get ATC from bootstrap samples for VRM-E
#step 3: Get ATC for naive bootstrap: Average(Boot_strap 2017 samples) - Average(2018 Empirical Samples
#step 4: Follow figure 1 from https://aclanthology.org/D12-1091.pdf
#Delta(Bootstrap) = ATC_VRM-E(Bootstrap) - ATC_SPSM(Boostrap)
#Delta(X) = ATC_VRM-E  - ATC_SPSM = -0.17 - -0.26 = 0.09

In [44]:
#data processing to get the propensity and strata
df_embeddings_2017 = pd.merge(df_embeddings_2017, df_paper_strata[df_paper_strata['treatment']==1][['paper_id','pscore','strata','treatment']], on='paper_id')

In [49]:
delta_x2 = 0.09
s2 = 0
B2 = 5000

for n in range(B2):
    bootstrap_sample = df_embeddings_2017.sample(n=df_embeddings_2017.shape[0], replace = True, random_state=n)
    ATC_VRM_E = bootstrap_sample.diff_2018_2017.mean()

    strata_2_weight = bootstrap_sample[(bootstrap_sample['strata'] ==2) & (bootstrap_sample['treatment'] ==1)].shape[0]/490
    strata_3_weight = bootstrap_sample[(bootstrap_sample['strata'] ==3) & (bootstrap_sample['treatment'] ==1)].shape[0]/490
    strata_4_weight = bootstrap_sample[(bootstrap_sample['strata'] ==4) & (bootstrap_sample['treatment'] ==1)].shape[0]/490

    strat_2_treated = bootstrap_sample[(bootstrap_sample['strata'] ==2) & (bootstrap_sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_2_control = df_paper_strata[(df_paper_strata['strata'] ==2) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()


    strat_3_treated = bootstrap_sample[(bootstrap_sample['strata'] ==3) & (bootstrap_sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_3_control = df_paper_strata[(df_paper_strata['strata'] ==3) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()

    strat_4_treated = bootstrap_sample[(bootstrap_sample['strata'] ==4) & (bootstrap_sample['treatment'] ==1)]['AVG_rating'].mean()
    strat_4_control = df_paper_strata[(df_paper_strata['strata'] ==4) & (df_paper_strata['treatment'] ==0)]['AVG_rating'].mean()



    ATC_SPSM = (strat_4_treated - strat_4_control)* strata_4_weight + (strat_3_treated - strat_3_control)* strata_3_weight + (strat_2_treated - strat_2_control)* strata_2_weight
    #treatment and control were flipped for SPSM procedure
    ATC_SPSM = -ATC_SPSM

    delta_bootstrap = ATC_VRM_E - ATC_SPSM

    if delta_bootstrap > 2 * delta_x2:
        s2 += 1
    
print('P-Value for VRM-E vs SPSM is: ',s2/B2)

P-Value is:  0.0
